In [ ]:
import sys
import logging

# Import Interactive Widgets for Jupyter Notebook (https://github.com/jupyter-widgets/ipywidgets)
# See https://ipywidgets.readthedocs.io for documentation on ui widgets
from ipywidgets import widgets

logging.root.level = logging.WARNING
# logging.root.level = logging.INFO

In [ ]:
# Upgrade pip because sbh-explorer requires version 18+ to process the synbiohub_adapter dependency
try:
    import pip
    pip_major = int(pip.__version__.split('.')[0])
    # Need pip to be at least version 18 to have PEP 508 dependency management URLs
    if pip_major < 18:
        raise ImportError()
except ImportError:
    # Install the dictionary writer library
    !{sys.executable} -m pip install --user --upgrade pip

In [ ]:
# Import SD2 Dictionary Writer, installing if necessary
try:
    from sd2_dictionary.sd2_dictionary_writer import SD2DictionaryWriter
except ImportError:
    # Install the dictionary writer library
    !{sys.executable} -m pip install --user --upgrade git+https://github.com/SD2E/sbol-dictionary-writer
    from sd2_dictionary.sd2_dictionary_writer import SD2DictionaryWriter

In [ ]:
# Install sbh-explorer
# !{sys.executable} -m pip install --user --upgrade git+https://github.com/SD2E/sbh-explorer@package

In [ ]:
import synbiohub_adapter as sbha
import sbh_explorer as sbhe

In [ ]:
# This may require authorization via google to get access to the google sheet representing the dictionary

dictionaryWriter = SD2DictionaryWriter(console=True)

In [ ]:
def make_filter():
    attrs = dictionaryWriter.dictionary.type_tabs.keys()
    attr_select = widgets.Dropdown(
        options=attrs,
        description='with '
    )
    hbox = widgets.HBox((attr_select, widgets.Label('[specialized]')))
    return hbox

def make_widgets():
    widget_box = widgets.VBox()

    sbh_types = dictionaryWriter.dictionary.type_tabs.keys()
    type_select = widgets.Dropdown(
        options=sbh_types,
        description='Look for '
    )

    add_filter = widgets.Button(
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Add a filter',
        icon='plus'
    )
    
    def add_filter_clicked(b):
        children = widget_box.children[:-1]
        add_btn = (widget_box.children[-1],)
        filter_box = make_filter()
        widget_box.children = children + (filter_box,) + add_btn

    add_filter.on_click(add_filter_clicked)
    widget_box.children = (widgets.HBox((type_select,)), add_filter)
    return widget_box

widget_box = make_widgets()
display(widget_box)

In [ ]:
# Dig the values out of the widgets above

look_for = widget_box.children[0].children[0].value
print('Looking for {}'.format(look_for))
for hbox in widget_box.children[1:-1]:
    attr = hbox.children[0].value
    print('with {}'.format(attr))

In [ ]:
# Authenticate to SynBioHub

# Adjust these values as needed
SBH_USER = 'scott'
SBH_PASSWORD = 'tiger'

sbh_query = sbha.SynBioHubQuery(sbha.SD2Constants.SD2_SERVER)
sbh_query.login(SBH_USER, SBH_PASSWORD)

# Upward Traversal

In [ ]:
mds = sbhe.parent_module_definitions(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1')
for md in mds:
    display(md)
    display(sbhe.parent_module_definitions(sbh_query, md))

In [ ]:
# Test for genetic construct
mds = sbhe.parent_module_definitions(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/YG_plasmid_002/1')
for md in mds:
    display(md)
    display(sbhe.parent_module_definitions(sbh_query, md))

# Downward traversal

In [ ]:
uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2etranscriptic0x2er1c7cpvfzqprk_r1c7fbvba55db_group_2/1'
# uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2euw_biofab0x2e10843_group_2/1'
children = sbhe.child_module_definitions(sbh_query, uri)
children

In [ ]:
for child in children:
    print(child)
    print('Child Module Definitions')
    print(sbhe.child_module_definitions(sbh_query, child))
    print('Child Component Definitions')
    child_cds = sbhe.child_component_definitions(sbh_query, child)
    print(child_cds)
    for cd in child_cds:
        display(sbhe.subject_info(sbh_query, cd))


In [ ]:
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_16967/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/NEB_10_beta_pAN3928_pAN4036_Larabinose_aTc_IPTG_system/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/CY9680x2DCY662int/1'
strain_uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783_Network/1'
sbhe.subject_info(sbh_query, strain_uri)

In [ ]:
sbhe.module_is_strain(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1')

In [ ]:
sbhe.module_is_strain(sbh_query, 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783_Network/1')

In [ ]:
# Look for parents of a given URI, then find strains within each of those
# This is the "used with" query
uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
uri = 'https://hub.sd2e.org/user/sd2e/design/CAT_R459942/1'
uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783_Network/1'
ancestors = list(sbhe.parent_module_definitions(sbh_query, uri))
roots = []
print(ancestors)
while ancestors:
    parent = ancestors.pop(0)
    print('Parent: {}'.format(parent))
    parents = sbhe.parent_module_definitions(sbh_query, parent)
    print('Found {} ancestors'.format(len(parents)))
    if not parents:
        roots.append(parent)
    ancestors.extend(parents)
    
for root in roots:
    strains = sbhe.find_contained_strains(sbh_query, root)
    print('{} strains: {}'.format(root, strains))

In [ ]:
sbhe.find_contained_strains(sbh_query, 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2etranscriptic0x2er1bbq4mr76ngd_group_2/1')